In [ ]:
# Part 1
# - Process Test
# - Clean Text
# - Tokenize the text and create sequences with Keras
def read_files(filepath):
    with open(filepath) as f:
        str_text = f.read()
    
    return str_text

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

In [ ]:
# Expand limit to file words count
nlp.max_length = 1198623

In [ ]:
def separate_punc(doc_text):
    return [
        token.text.lower()
        for token in nlp(doc_text) 
        if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n '
    ]

In [ ]:
d = read_files('moby_dick_four_chapters.txt')

In [ ]:
tokens = separate_punc(d)

In [ ]:
len(tokens)

In [ ]:
# 25 words --> network predict #26

In [ ]:
train_len = 25 + 1

text_sequences = []

for i in range(train_len, len(tokens)):
    seq = tokens[i - train_len : i]
    
    text_sequences.append(seq)

In [ ]:
' '.join(text_sequences[0])

In [ ]:
# Notice it is just one token over
' '.join(text_sequences[1])

In [ ]:
# Notice it is just one token over
' '.join(text_sequences[2])

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [ ]:
# Tokenization of the texts
sequences = tokenizer.texts_to_sequences(text_sequences)

In [ ]:
for i in sequences[0]:
    print(f'{i}: {tokenizer.index_word[i]}')

In [ ]:
vocabulary_size = len(tokenizer.word_counts)

In [ ]:
vocabulary_size

In [ ]:
import numpy as np

In [ ]:
sequences = np.array(sequences)

In [ ]:
sequences

In [ ]:
# Part 2
# - Create the LSTM Based Model
# - Split the Data into Features and Labels
# - X Features (First n words of sequence)
# - Y Label (Next word after the sequence)
# Fit the Model

In [ ]:
from keras.utils import to_categorical

In [ ]:
# Grab all columns, except the last column
X = sequences[:,:-1]

In [ ]:
# Grab only the last column
y = sequences[:, -1]

In [ ]:
y = to_categorical(y, num_classes=vocabulary_size + 1)

In [ ]:
seq_len = X.shape[1]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [ ]:
def create_model(vocabulary_size, seq_len):

    model = Sequential()

    model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
    
    return model

In [ ]:
model = create_model(vocabulary_size + 1, seq_len)

In [ ]:
from pickle import dump, load

In [ ]:
model.fit(X, y, batch_size=128, epochs=2, verbose=1)

In [ ]:
model.save('my_mobydick_model.h5')

In [ ]:
dump(tokenizer, open('my_simpletokenizer', 'wb'))